In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

import tensorflow_hub as hub
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model

In [14]:
import os
import random
import shutil

dataset_path = "C:\\Users\\Rayaan_Ghosh\\Desktop\\spechio-face\\ML\\skin-dataset"
oily_path = os.path.join(dataset_path , "oily")
dry_path = os.path.join(dataset_path , "dry")
subfolder_paths = [os.path.join(dataset_path , "train") ,os.path.join(dataset_path , "test") ]

train_ratio = 0.8

for path in subfolder_paths:
    os.makedirs(path)
    os.makedirs(os.path.join(path,"oily"), exist_ok=True) #skin_dataset/train/oily  , skin_dataset/test/oily
    os.makedirs(os.path.join(path,"dry") , exist_ok=True) #skin_dataset/train/dry , skin_dataset/test/dry

for p in [oily_path , dry_path]:
    files = os.listdir(p)
    random.shuffle(files)

    split_index = int(train_ratio * len(files))
    train_files = files[:split_index]
    test_files = files[split_index:]
    directory, filename = os.path.split(p)
    for file in train_files:
        file_path = os.path.join(p, file)
        destination_path = os.path.join(os.path.join(subfolder_paths[0] , filename ), file)
        shutil.move(file_path, destination_path)

    # Move the test files to the test subfolder
    for file in test_files:
        file_path = os.path.join(p, file)
        destination_path = os.path.join(os.path.join(subfolder_paths[1] , filename ), file)
        shutil.move(file_path, destination_path)
    
        
    


In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Preprocess data (get all of the pixel values between 0 & 1, also called scaling/normalization)
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)

# Setup paths to our data directories
train_dir = subfolder_paths[0]
test_dir = subfolder_paths[1]

# Import data from directories and turn it into batches
train_data = train_datagen.flow_from_directory(directory=train_dir,
                                               target_size=(224, 224),
                                               class_mode="categorical",
                                               seed=42)
valid_data = valid_datagen.flow_from_directory(directory=test_dir,
                                               target_size=(224, 224),
                                               class_mode="categorical",
                                               seed=42)

Found 411 images belonging to 2 classes.
Found 103 images belonging to 2 classes.


In [21]:
efficientnet_url = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"
IMAGE_SHAPE = (224, 224)
BATCH_SIZE = 32

def create_model(model_url, num_classes=3):
    
    """
    Takes a TensorFlow Hub URL and creates a Keras Sequential model with it.

    Args:
    model_url(str): A TensorFlow Hub feature extraction URL.
    num_classes(int): Number of output neurons in the output layer, 
      should be equal to number of target classes, default = 10

    Returns:
    An uncompiled Keras Sequential model with model_url as feature extractor 
    layer and Dense output layer with num_classes output neurons.
    """
    # Download the pretrained model and save it as a Keras layer
    feature_extractor_layer = hub.KerasLayer(model_url,
                                           trainable = False,   # freeze the already learned patterns
                                           name="EfficientnetB0_model",
                                           input_shape=IMAGE_SHAPE+(3,))  # define the input image shape
    # Create our image model
    model = tf.keras.Sequential([   
        feature_extractor_layer,    # use the feature extraction layer as the base
        layers.Dense(num_classes, activation="softmax", name="output_layer")   # create our own output layer
    ])
    return model

In [23]:
# Create EfficientNet model
efficientnet_model = create_model(efficientnet_url,
                            num_classes=2)
# Compile
efficientnet_model.compile(loss="categorical_crossentropy",
                     optimizer=tf.keras.optimizers.Adam(),
                     metrics=["accuracy"])

In [24]:
efficientnet_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 EfficientnetB0_model (Keras  (None, 1280)             4049564   
 Layer)                                                          
                                                                 
 output_layer (Dense)        (None, 2)                 2562      
                                                                 
Total params: 4,052,126
Trainable params: 2,562
Non-trainable params: 4,049,564
_________________________________________________________________


In [25]:
# Fit the model
efficientnet_history = efficientnet_model.fit(train_data,
                                  epochs=5,
                                  validation_data=valid_data)   # name of log files


Epoch 1/5
13/13 [==============================] - 75s 3s/step - loss: 0.3539 - accuracy: 0.8613 - val_loss: 0.2576 - val_accuracy: 0.8738
Epoch 2/5
13/13 [==============================] - 34s 3s/step - loss: 0.1914 - accuracy: 0.9562 - val_loss: 0.1805 - val_accuracy: 0.9417
Epoch 3/5
13/13 [==============================] - 34s 3s/step - loss: 0.1445 - accuracy: 0.9611 - val_loss: 0.1495 - val_accuracy: 0.9515
Epoch 4/5
13/13 [==============================] - 35s 3s/step - loss: 0.1208 - accuracy: 0.9708 - val_loss: 0.1324 - val_accuracy: 0.9515
Epoch 5/5
13/13 [==============================] - 36s 3s/step - loss: 0.1079 - accuracy: 0.9757 - val_loss: 0.1191 - val_accuracy: 0.9612


In [26]:
efficientnet_model.save('saved_model/my_model')

INFO:tensorflow:Assets written to: saved_model/my_model\assets


INFO:tensorflow:Assets written to: saved_model/my_model\assets


In [2]:
model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 EfficientnetB0_model (Keras  (None, 1280)             4049564   
 Layer)                                                          
                                                                 
 output_layer (Dense)        (None, 2)                 2562      
                                                                 
Total params: 4,052,126
Trainable params: 2,562
Non-trainable params: 4,049,564
_________________________________________________________________


In [3]:
train_dir = "C:\\Users\\Rayaan_Ghosh\\Desktop\\spechio-face\\ML\\skin-dataset\\train"
import pathlib 
data_dir = pathlib.Path(train_dir)
class_names = np.array(sorted([item.name for item in data_dir.glob('*')]))
print(class_names)

['dry' 'oily']


In [4]:
# Create a function to import an image and resize it to be able to be used with our model
def load_and_prep_image(filename, img_shape=224):
    """
    Reads in an image from filename, turns it into a tensor and reshapes into (224,224,3).
    """
    # Read in the image
    img = tf.io.read_file(filename)
    # Decode it into a tensor
    img = tf.image.decode_jpeg(img)
    # Resize the image
    img = tf.image.resize(img, [img_shape, img_shape])
    # Rescale the image (get all values between 0 and 1)
    img = img/255.
    return img

In [5]:
# Reconfig pred_and_plot function to work with multi-class images
def pred_and_plot(model, filename, class_names=class_names):
    """
    Imports an image located at filename, makes a prediction with model
    and plots the image with the predicted class as the title.
    """
    # Import the target image and preprocess it
    img = load_and_prep_image(filename)

    # Make a prediction
    pred = model.predict(tf.expand_dims(img, axis=0))

    # Add in logic for multi-class & get pred_class name
    if len(pred[0]) > 1:
        pred_class = class_names[tf.argmax(pred[0])]
    else:
        pred_class = class_names[int(tf.round(pred[0]))]

    print('Prediction Probabilities : ', pred[0])

    # Plot the image and predicted class
    plt.imshow(img)
    plt.title(f"Prediction: {pred_class}")
    plt.axis(False);

In [ ]:
img1 = "C:\\Users\\Rayaan_Ghosh\\Desktop\\spechio-face\\ML\\skin-tone\\test_imgs\\test_1.jpg"
img2 = "C:\\Users\\Rayaan_Ghosh\\Pictures\\Camera Roll\\WIN_20230629_14_33_03_Pro.jpg"

pred_and_plot(model=model, 
              filename=img1, 
              class_names=class_names)

1/1 [==============================] - 2s 2s/step
Prediction Probabilities :  [0.9504036  0.04959644]
